In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0762,  0.0202,  0.0889,  0.0647, -0.1032, -0.1496,  0.0864,  0.0230,
         -0.0829,  0.0948]], grad_fn=<AddmmBackward>)


In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

tensor(0.8663, grad_fn=<MseLossBackward>)


In [31]:
 # a dummy target, for example

In [36]:
target = torch.ones(10) 

In [54]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
print(net.conv2.bias.grad)

net.conv2.bias.requires_grad = True
net.conv2.weight.requires_grad = True

output = net(input)
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()
print(loss)
loss = criterion(output, target)
loss.backward(retain_graph = True)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)
print(net.conv2.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor(1.0143, grad_fn=<MseLossBackward>)
conv1.bias.grad after backward
tensor([-0.0155,  0.0047, -0.0127,  0.0032, -0.0125,  0.0271])
tensor([ 0.0053,  0.0000, -0.0040,  0.0088, -0.0061, -0.0127, -0.0012,  0.0005,
        -0.0293, -0.0258, -0.0005,  0.0032, -0.0034,  0.0101,  0.0370, -0.0086])


In [50]:
[-0.0155,  0.0047, -0.0127,  0.0032, -0.0125,  0.0271]

[-0.0155, 0.0047, -0.0127, 0.0032, -0.0125, 0.0271]

In [ ]:
[-0.0062,  0.0278, -0.0228,  0.0244,  0.0170, -0.0259])

In [7]:
x = torch.ones(2,2, requires_grad = True)
y = x+2
z = y.mean()
z.backward()

In [10]:
x.grad.data

tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])